In [5]:
from pathlib import Path
from collections import defaultdict

from sklearn.cluster import KMeans
from sklearn.preprocessing import MultiLabelBinarizer

In [6]:
def get_tokenised_genes_from_match_line(match_line):
    tokenised_genes = list()
    match = match_line.split()[2]
    for gene_and_topic_prob in match.split(","):
        gene, gene_to_topic_prob = gene_and_topic_prob.split(":")
        tokenised_genes.append(gene)
    return tokenised_genes

In [7]:
def get_module_matches(input_file):
    module_matches = defaultdict(list)
    with open(input_file, "r") as infile:
        for line in infile: 
            # skip header lines
            if line.startswith("#"): 
                continue
            tokenised_genes = get_tokenised_genes_from_match_line(line)
            if len(tokenised_genes) < 2:
                # skip matches with less than 2 (unique) genes
                continue
            bgc_id = line.split()[3]
            module_matches[tuple(tokenised_genes)].append(bgc_id)
    return module_matches

In [8]:
input_file = Path("../data/matches_per_topic_filtered.txt")
module_matches = get_module_matches(input_file)

module_matches


defaultdict(list,
            {('OTCace_N;OTCace',
              'Alpha-amylase;DUF3459'): ['NZ_JPMW01000001.cluster015'],
             ('OTCace_N;OTCace',
              'Amidohydro_1'): ['NZ_ATVG01000001.cluster002',
              'NC_013173.1.cluster001',
              'NZ_CP014673.1.cluster002',
              'NZ_AUAR01000001.cluster003',
              'NZ_CP014672.1.cluster002',
              'NZ_FQWW01000019.cluster007',
              'NZ_FORX01000044.cluster001',
              'NZ_AUDH01000001.cluster002',
              'NZ_FOTO01000043.cluster002',
              'NZ_CDSB01000001.cluster009',
              'NZ_FNNQ01000040.cluster012'],
             ('OTCace_N;OTCace', 'FGGY_N;FGGY_C'): ['NZ_CM001165.1.cluster017',
              'NZ_FMCY01000191.cluster003',
              'NZ_CP013002.1.cluster002',
              'NZ_FNJR01000034.cluster010'],
             ('OTCace_N;OTCace',
              'Glyco_hydro_20b;Glyco_hydro_20'): ['NZ_KB903813.1.cluster020', 'NZ_KB899659.1.cluster001']

In [9]:
top_modules = list(module_matches.keys())

print(f"Found {len(top_modules)} top modules in the dataset.")
print(f"Example top module: {top_modules[0]} with {len(module_matches[top_modules[0]])} BGCs.")

Found 33582 top modules in the dataset.
Example top module: ('OTCace_N;OTCace', 'Alpha-amylase;DUF3459') with 1 BGCs.


In [10]:
def kmeans_clustering(modules, n_clusters):
    """
    Perform KMeans clustering on the subcluster matches.
    """
    mlb = MultiLabelBinarizer()
    X = mlb.fit_transform(modules)
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(X)
    
    return kmeans.labels_

In [11]:
num_clusters = 600  # Example number of clusters
labels = kmeans_clustering(top_modules, n_clusters=num_clusters)

In [12]:
modules_per_label = defaultdict(list)
for module, label in zip(top_modules, labels):
    modules_per_label[label].append(module)

# Example: print the number of modules in each cluster
for label, modules in modules_per_label.items():
    print(f"Cluster {label}: {len(modules)} modules")

Cluster 519: 36 modules
Cluster 397: 94 modules
Cluster 544: 3204 modules
Cluster 340: 18 modules
Cluster 21: 479 modules
Cluster 56: 362 modules
Cluster 15: 418 modules
Cluster 2: 563 modules
Cluster 100: 256 modules
Cluster 143: 343 modules
Cluster 517: 23 modules
Cluster 149: 257 modules
Cluster 160: 93 modules
Cluster 17: 746 modules
Cluster 327: 16 modules
Cluster 104: 648 modules
Cluster 7: 269 modules
Cluster 260: 144 modules
Cluster 215: 244 modules
Cluster 16: 343 modules
Cluster 86: 177 modules
Cluster 572: 48 modules
Cluster 68: 157 modules
Cluster 488: 28 modules
Cluster 514: 50 modules
Cluster 36: 48 modules
Cluster 102: 89 modules
Cluster 9: 55 modules
Cluster 74: 244 modules
Cluster 422: 135 modules
Cluster 31: 601 modules
Cluster 265: 222 modules
Cluster 287: 52 modules
Cluster 45: 149 modules
Cluster 27: 335 modules
Cluster 37: 310 modules
Cluster 536: 88 modules
Cluster 11: 358 modules
Cluster 61: 345 modules
Cluster 214: 315 modules
Cluster 43: 397 modules
Cluster 26

In [13]:
output_file = Path(f"../results/top_modules_k{num_clusters}.txt")

with open(output_file, "w") as f:
    for label in sorted(modules_per_label.keys()):
        modules = modules_per_label[label]
        f.write(f"#model: {label}, matches: {len(modules)}\n")
        for module in sorted(modules):
            bgc_ids = module_matches[module]
            for bgc_id in bgc_ids:
                f.write(f"{bgc_id}\t{','.join(module)}\n")